In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    param1=["best","random"]
    score=0
    best_param=param1[0]
    for i in param1:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        if ACC>score:
            score=ACC
            best_param=i
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param1,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
    return best_param,results

In [5]:
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    score=0
    best_param=param2[0]
    #param1=["best","random"]
    for i in param2:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        if ACC>score:
            score=ACC
            best_param=i
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param2,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
  #  return grid_search.best_params_["splitter"],results
    return best_param,results

In [6]:
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,100))  ## 设置max_features范围
  #  param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    #param1=["best","random"]
    for j in param3:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=j)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param3,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    return results

In [8]:
file_path = 'E:\datatest\PseDPC\d8\d81'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
811.csv
**************************************************
2022-02-24 18:39:50.237006
2022-02-24 18:39:56.065410
2022-02-24 18:40:24.942151
##################################################
812.csv
**************************************************
2022-02-24 18:40:41.801044
2022-02-24 18:40:47.176662
2022-02-24 18:41:13.305753
##################################################
813.csv
**************************************************
2022-02-24 18:41:28.293653
2022-02-24 18:41:33.546602
2022-02-24 18:41:59.739523
##################################################
814.csv
**************************************************
2022-02-24 18:42:15.444501
2022-02-24 18:42:20.415205
2022-02-24 18:42:45.104148
##################################################
815.csv
**************************************************
2022-02-24 18:43:02.903525
2022-02-24 18:43:07.992909
2022-02-24 18:43:16.514111
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d8\d82'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
821.csv
**************************************************
2022-02-24 18:45:15.695243
2022-02-24 18:45:21.295260
2022-02-24 18:45:47.299684
##################################################
822.csv
**************************************************
2022-02-24 18:46:05.552848
2022-02-24 18:46:10.585384
2022-02-24 18:46:35.280313
##################################################
823.csv
**************************************************
2022-02-24 18:46:49.788496
2022-02-24 18:46:55.195031
2022-02-24 18:47:22.465070
##################################################
824.csv
**************************************************
2022-02-24 18:47:38.712600
2022-02-24 18:47:44.727506
2022-02-24 18:48:14.295401
##################################################
825.csv
**************************************************
2022-02-24 18:48:31.076500
2022-02-24 18:48:37.105371
2022-02-24 18:49:06.362103
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d8\d83'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
831.csv
**************************************************
2022-02-24 18:52:37.666750
2022-02-24 18:52:42.521942
2022-02-24 18:53:06.251271
##################################################
832.csv
**************************************************
2022-02-24 18:53:23.868643
2022-02-24 18:53:29.115606
2022-02-24 18:53:53.871372
##################################################
833.csv
**************************************************
2022-02-24 18:54:08.936065
2022-02-24 18:54:13.994535
2022-02-24 18:54:38.699434
##################################################
834.csv
**************************************************
2022-02-24 18:54:54.034406
2022-02-24 18:54:59.272419
2022-02-24 18:55:27.773138
##################################################
835.csv
**************************************************
2022-02-24 18:55:43.584293
2022-02-24 18:55:49.261105
2022-02-24 18:56:16.419443
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d8\d84'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
841.csv
**************************************************
2022-02-24 18:59:00.683154
2022-02-24 18:59:05.844299
2022-02-24 18:59:16.472863
##################################################
842.csv
**************************************************
2022-02-24 18:59:26.050755
2022-02-24 18:59:31.533086
2022-02-24 18:59:57.931457
##################################################
843.csv
**************************************************
2022-02-24 19:00:16.918658
2022-02-24 19:00:22.754044
2022-02-24 19:00:54.392872
##################################################
844.csv
**************************************************
2022-02-24 19:01:16.222466
2022-02-24 19:01:22.285245
2022-02-24 19:01:51.380402
##################################################
845.csv
**************************************************
2022-02-24 19:02:13.977493
2022-02-24 19:02:19.982426
2022-02-24 19:02:51.061827
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d8\d85'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
851.csv
**************************************************
2022-02-24 19:05:21.283394
2022-02-24 19:05:27.013065
2022-02-24 19:05:36.044900
##################################################
852.csv
**************************************************
2022-02-24 19:05:43.741714
2022-02-24 19:05:49.653897
2022-02-24 19:05:59.373892
##################################################
853.csv
**************************************************
2022-02-24 19:06:08.020711
2022-02-24 19:06:14.244059
2022-02-24 19:06:23.474364
##################################################
854.csv
**************************************************
2022-02-24 19:06:32.304739
2022-02-24 19:06:38.346574
2022-02-24 19:07:09.110266
##################################################
855.csv
**************************************************
2022-02-24 19:07:26.359116
2022-02-24 19:07:32.872689
2022-02-24 19:07:42.032183
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d8\d86'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
861.csv
**************************************************
2022-02-24 19:10:27.816089
2022-02-24 19:10:33.221627
2022-02-24 19:11:00.311151
##################################################
862.csv
**************************************************
2022-02-24 19:11:18.723396
2022-02-24 19:11:24.650538
2022-02-24 19:11:54.374013
##################################################
863.csv
**************************************************
2022-02-24 19:12:10.669414
2022-02-24 19:12:16.257464
2022-02-24 19:12:46.284127
##################################################
864.csv
**************************************************
2022-02-24 19:13:03.166957
2022-02-24 19:13:09.025285
2022-02-24 19:13:37.685603
##################################################
865.csv
**************************************************
2022-02-24 19:14:01.119916
2022-02-24 19:14:06.943324
2022-02-24 19:14:37.032822
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d8\d87'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
871.csv
**************************************************
2022-02-24 19:18:47.578189
2022-02-24 19:18:52.811189
2022-02-24 19:19:17.347562
##################################################
872.csv
**************************************************
2022-02-24 19:19:32.985701
2022-02-24 19:19:38.364311
2022-02-24 19:20:04.642005
##################################################
873.csv
**************************************************
2022-02-24 19:20:19.850315
2022-02-24 19:20:25.018488
2022-02-24 19:20:51.082753
##################################################
874.csv
**************************************************
2022-02-24 19:21:09.196291
2022-02-24 19:21:14.404356
2022-02-24 19:21:41.129853
##################################################
875.csv
**************************************************
2022-02-24 19:22:01.276949
2022-02-24 19:22:06.806156
2022-02-24 19:22:34.149996
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d8\d88'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
881.csv
**************************************************
2022-02-24 19:26:23.553230
2022-02-24 19:26:28.996667
2022-02-24 19:26:56.056268
##################################################
882.csv
**************************************************
2022-02-24 19:27:11.584757
2022-02-24 19:27:16.533503
2022-02-24 19:27:42.134991
##################################################
883.csv
**************************************************
2022-02-24 19:28:02.197308
2022-02-24 19:28:08.920321
2022-02-24 19:28:37.269473
##################################################
884.csv
**************************************************
2022-02-24 19:28:54.041599
2022-02-24 19:28:59.435169
2022-02-24 19:29:09.012547
##################################################
885.csv
**************************************************
2022-02-24 19:29:17.791060
2022-02-24 19:29:23.952573
2022-02-24 19:29:35.826803
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d8\d89'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
891.csv
**************************************************
2022-02-24 19:31:24.206833
2022-02-24 19:31:29.947476
2022-02-24 19:31:58.276253
##################################################
892.csv
**************************************************
2022-02-24 19:32:18.521089
2022-02-24 19:32:24.803280
2022-02-24 19:32:57.129790
##################################################
893.csv
**************************************************
2022-02-24 19:33:14.997985
2022-02-24 19:33:22.312416
2022-02-24 19:33:54.471374
##################################################
894.csv
**************************************************
2022-02-24 19:34:11.851894
2022-02-24 19:34:17.388082
2022-02-24 19:34:45.246526
##################################################
895.csv
**************************************************
2022-02-24 19:35:00.661315
2022-02-24 19:35:06.257311
2022-02-24 19:35:15.171494
#############################################

In [17]:
file_path = 'E:\datatest\PseDPC\d8\d810'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8101.csv
**************************************************
2022-02-24 19:37:38.416210
2022-02-24 19:37:44.158849
2022-02-24 19:38:12.649620
##################################################
8102.csv
**************************************************
2022-02-24 19:38:32.045725
2022-02-24 19:38:37.717550
2022-02-24 19:38:47.389673
##################################################
8103.csv
**************************************************
2022-02-24 19:38:54.526577
2022-02-24 19:39:00.262234
2022-02-24 19:39:28.522622
##################################################
8104.csv
**************************************************
2022-02-24 19:39:43.905465
2022-02-24 19:39:49.542383
2022-02-24 19:39:58.126445
##################################################
8105.csv
**************************************************
2022-02-24 19:40:05.835791
2022-02-24 19:40:11.468720
2022-02-24 19:40:19.945041
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d8\d811'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8111.csv
**************************************************
2022-02-24 19:43:18.301849
2022-02-24 19:43:23.524874
2022-02-24 19:43:49.520323
##################################################
8112.csv
**************************************************
2022-02-24 19:44:07.012523
2022-02-24 19:44:12.642461
2022-02-24 19:44:21.233477
##################################################
8113.csv
**************************************************
2022-02-24 19:44:28.857100
2022-02-24 19:44:34.285555
2022-02-24 19:44:42.787807
##################################################
8114.csv
**************************************************
2022-02-24 19:44:49.850910
2022-02-24 19:44:55.278410
2022-02-24 19:45:03.815548
##################################################
8115.csv
**************************************************
2022-02-24 19:45:10.905578
2022-02-24 19:45:16.289176
2022-02-24 19:45:42.714473
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d8\d812'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8121.csv
**************************************************
2022-02-24 19:47:29.470849
2022-02-24 19:47:35.597456
2022-02-24 19:48:04.709566
##################################################
8122.csv
**************************************************
2022-02-24 19:48:20.071466
2022-02-24 19:48:25.829061
2022-02-24 19:48:54.160263
##################################################
8123.csv
**************************************************
2022-02-24 19:49:15.523105
2022-02-24 19:49:21.255767
2022-02-24 19:49:49.391490
##################################################
8124.csv
**************************************************
2022-02-24 19:50:04.502062
2022-02-24 19:50:10.337450
2022-02-24 19:50:39.331875
##################################################
8125.csv
**************************************************
2022-02-24 19:50:54.485332
2022-02-24 19:51:00.094326
2022-02-24 19:51:28.065521
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d8\d813'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8131.csv
**************************************************
2022-02-24 19:54:16.599577
2022-02-24 19:54:22.441946
2022-02-24 19:54:51.309732
##################################################
8132.csv
**************************************************
2022-02-24 19:55:06.594814
2022-02-24 19:55:12.990702
2022-02-24 19:55:21.738324
##################################################
8133.csv
**************************************************
2022-02-24 19:55:29.430717
2022-02-24 19:55:35.611207
2022-02-24 19:55:44.389695
##################################################
8134.csv
**************************************************
2022-02-24 19:55:51.885640
2022-02-24 19:55:58.060147
2022-02-24 19:56:06.889497
##################################################
8135.csv
**************************************************
2022-02-24 19:56:14.550001
2022-02-24 19:56:20.378407
2022-02-24 19:56:29.173875
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d8\d814'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8141.csv
**************************************************
2022-02-24 19:58:06.949300
2022-02-24 19:58:12.874457
2022-02-24 19:58:41.772110
##################################################
8142.csv
**************************************************
2022-02-24 19:59:00.184846
2022-02-24 19:59:05.692114
2022-02-24 19:59:32.913314
##################################################
8143.csv
**************************************************
2022-02-24 19:59:50.375560
2022-02-24 19:59:56.212978
2022-02-24 20:00:25.384893
##################################################
8144.csv
**************************************************
2022-02-24 20:00:43.756742
2022-02-24 20:00:49.892327
2022-02-24 20:00:59.043860
##################################################
8145.csv
**************************************************
2022-02-24 20:01:06.877913
2022-02-24 20:01:12.895782
2022-02-24 20:01:21.979508
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d8\d815'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8151.csv
**************************************************
2022-02-24 20:02:57.928763
2022-02-24 20:03:03.427080
2022-02-24 20:03:31.039177
##################################################
8152.csv
**************************************************
2022-02-24 20:03:45.407755
2022-02-24 20:03:50.937965
2022-02-24 20:04:17.339301
##################################################
8153.csv
**************************************************
2022-02-24 20:04:31.940264
2022-02-24 20:04:37.540290
2022-02-24 20:05:05.571256
##################################################
8154.csv
**************************************************
2022-02-24 20:05:23.006608
2022-02-24 20:05:28.675469
2022-02-24 20:05:56.699463
##################################################
8155.csv
**************************************************
2022-02-24 20:06:11.271509
2022-02-24 20:06:16.727877
2022-02-24 20:06:24.968830
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d8\d816'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8161.csv
**************************************************
2022-02-24 20:09:52.445272
2022-02-24 20:09:57.899679
2022-02-24 20:10:25.203632
##################################################
8162.csv
**************************************************
2022-02-24 20:10:44.179048
2022-02-24 20:10:50.428328
2022-02-24 20:11:21.199999
##################################################
8163.csv
**************************************************
2022-02-24 20:11:36.924927
2022-02-24 20:11:42.853067
2022-02-24 20:12:15.601449
##################################################
8164.csv
**************************************************
2022-02-24 20:12:32.472310
2022-02-24 20:12:38.661772
2022-02-24 20:13:08.741274
##################################################
8165.csv
**************************************************
2022-02-24 20:13:24.587875
2022-02-24 20:13:30.433240
2022-02-24 20:14:00.842387
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d8\d817'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8171.csv
**************************************************
2022-02-24 20:17:56.440154
2022-02-24 20:18:01.954402
2022-02-24 20:18:29.691192
##################################################
8172.csv
**************************************************
2022-02-24 20:18:44.914462
2022-02-24 20:18:50.643135
2022-02-24 20:19:20.618935
##################################################
8173.csv
**************************************************
2022-02-24 20:19:38.468179
2022-02-24 20:19:44.223782
2022-02-24 20:20:13.528909
##################################################
8174.csv
**************************************************
2022-02-24 20:20:29.710615
2022-02-24 20:20:35.908034
2022-02-24 20:21:06.247860
##################################################
8175.csv
**************************************************
2022-02-24 20:21:21.703538
2022-02-24 20:21:27.842085
2022-02-24 20:21:59.807582
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d8\d818'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8181.csv
**************************************************
2022-02-24 20:25:52.408058
2022-02-24 20:25:59.337518
2022-02-24 20:26:30.279754
##################################################
8182.csv
**************************************************
2022-02-24 20:26:47.927516
2022-02-24 20:26:53.936439
2022-02-24 20:27:24.915557
##################################################
8183.csv
**************************************************
2022-02-24 20:27:46.759205
2022-02-24 20:27:53.046384
2022-02-24 20:28:26.443031
##################################################
8184.csv
**************************************************
2022-02-24 20:28:47.350096
2022-02-24 20:28:53.660234
2022-02-24 20:29:27.585457
##################################################
8185.csv
**************************************************
2022-02-24 20:29:50.511108
2022-02-24 20:29:57.722834
2022-02-24 20:30:32.173192
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d8\d819'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8191.csv
**************************************************
2022-02-24 20:34:47.777328
2022-02-24 20:34:53.205800
2022-02-24 20:35:21.002458
##################################################
8192.csv
**************************************************
2022-02-24 20:35:36.757278
2022-02-24 20:35:42.175781
2022-02-24 20:36:08.741704
##################################################
8193.csv
**************************************************
2022-02-24 20:36:27.893493
2022-02-24 20:36:35.405366
2022-02-24 20:37:04.569337
##################################################
8194.csv
**************************************************
2022-02-24 20:37:21.034286
2022-02-24 20:37:26.722068
2022-02-24 20:37:55.689567
##################################################
8195.csv
**************************************************
2022-02-24 20:38:13.735962
2022-02-24 20:38:20.045171
2022-02-24 20:38:51.044233
########################################

In [27]:
file_path = 'E:\datatest\PseDPC\d8\d820'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
8201.csv
**************************************************
2022-02-24 20:42:49.216181
2022-02-24 20:42:54.615735
2022-02-24 20:43:21.567625
##################################################
8202.csv
**************************************************
2022-02-24 20:43:41.982006
2022-02-24 20:43:48.143111
2022-02-24 20:44:17.340991
##################################################
8203.csv
**************************************************
2022-02-24 20:44:37.157998
2022-02-24 20:44:42.766004
2022-02-24 20:44:51.764890
##################################################
8204.csv
**************************************************
2022-02-24 20:44:58.073013
2022-02-24 20:45:03.593243
2022-02-24 20:45:13.353673
##################################################
8205.csv
**************************************************
2022-02-24 20:45:20.454702
2022-02-24 20:45:26.482546
2022-02-24 20:45:35.562254
########################################